In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
# 全てのネットワークはnn.Moduleを継承している，

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# flatten()は，[28, 28]のイメージを28x28のサイズの配列にしている．
# minibatchの次元(最初のやつ)は保持されたままになる．
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [9]:
# nn.Linearは一般的なニューラルネットワークの層だと思っておけば良い
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [10]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0200, -0.2509, -0.1581, -0.1591,  0.1997, -0.1514,  0.6181, -0.1749,
         -0.5678, -0.1233,  0.1484, -0.1968, -0.0568, -0.1454,  0.3626,  0.4333,
         -0.1358,  0.1738, -0.4019,  0.0786],
        [ 0.1666, -0.0415,  0.0073, -0.4191,  0.0570, -0.0108,  0.5805, -0.5508,
         -0.6327, -0.0918, -0.1300, -0.1069,  0.0586, -0.3818,  0.0627,  1.0431,
          0.1997,  0.2434, -0.5889,  0.1428],
        [ 0.2516, -0.1683, -0.2927, -0.1184,  0.1596, -0.1536,  0.5045, -0.4758,
         -0.3005, -0.0495,  0.0535, -0.2458, -0.0527,  0.0924,  0.2703,  0.7754,
          0.0101,  0.3199, -0.1892,  0.0880]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.1997, 0.0000, 0.6181, 0.0000, 0.0000,
         0.0000, 0.1484, 0.0000, 0.0000, 0.0000, 0.3626, 0.4333, 0.0000, 0.1738,
         0.0000, 0.0786],
        [0.1666, 0.0000, 0.0073, 0.0000, 0.0570, 0.0000, 0.5805, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0586, 0.0000, 0.06

In [11]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [13]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0981, 0.1201, 0.0766, 0.1083, 0.0639, 0.1070, 0.1024, 0.1172, 0.1243,
         0.0821],
        [0.0992, 0.1219, 0.0809, 0.1037, 0.0604, 0.1104, 0.0885, 0.1219, 0.1310,
         0.0821],
        [0.0952, 0.1174, 0.0811, 0.1034, 0.0630, 0.1189, 0.0891, 0.1179, 0.1160,
         0.0981]], grad_fn=<SoftmaxBackward0>)

In [14]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0177, -0.0074, -0.0110,  ..., -0.0179, -0.0158,  0.0141],
        [ 0.0232, -0.0214,  0.0143,  ..., -0.0171, -0.0131, -0.0308]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0130, 0.0349], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0299, -0.0401,  0.0252,  ..., -0.0019,  0.0199, -0.0001],
        [-0.0278, -0.0062, -0.0350,  ...,  0.0051, -0.0277,  0.0149]],
       device='cuda:0', grad_fn=<Slic